# technicals

> This module implements various methods for adding several technical indicators and metrics to the raw bhavcopy data.

In [ ]:
# | default_exp technicals


In [ ]:
# | hide
from nbdev.showdoc import *


In [ ]:
# | export
import pandas as pd
import os
import numpy as np
from pathlib import Path
import pandas_ta as pdta
import nbdev
import shutil
import datetime

from stocksurfer.scrapers import fetch_bhavcopy_data_for_range

In [ ]:
# | export
base_path = nbdev.config.get_config().lib_path
raw_data_dir = base_path / "../Data/Bhavcopy/Raw"
processed_data_dir = base_path / "../Data/Bhavcopy/Processed"

## Bhavcopy Utils

Load multiple bhavcopy files and concatenate them into a single dataframe.

In [ ]:
def load_multiple_bhavcopy(files_to_load):
    
    bhavcopy_dtypes = {
        "SYMBOL": "string",
        "SERIES": "string",
        "OPEN": "float64",
        "HIGH": "float64",
        "LOW": "float64",
        "CLOSE": "float64",
        "TOTTRDQTY": "int64",
        "TOTTRDVAL": "float64",
        "TIMESTAMP": "string",
        "TOTALTRADES": "int64",
        # "ISIN": 'string',
        # "Unnamed: 13": 'string',
    }

    bhavcopy_usecols = [
        "SYMBOL",
        "SERIES",
        "OPEN",
        "HIGH",
        "LOW",
        "CLOSE",
        "TOTTRDQTY",
        "TOTTRDVAL",
        "TIMESTAMP",
        "TOTALTRADES",
    ]
    
    return pd.concat(
            [
                pd.read_csv(
                    f,
                    dtype=bhavcopy_dtypes,
                    usecols=bhavcopy_usecols,
                    parse_dates=["TIMESTAMP"],
                    dayfirst=False,
                )
                for f in files_to_load
            ],
            ignore_index=True,
    )

Get all raw bhavcopy data

In [ ]:
# | export
def get_raw_bhavcopy_data(start_date: datetime=None, end_date:datetime.datetime=None) -> pd.DataFrame:
    
    if start_date:
        end_date = end_date or datetime.datetime.today()
        # Get list of date from bhavcopy_date till today
        date_list = pd.date_range(start_date, end_date).tolist()
        
        files_to_load = []
        for d in date_list:
            # Get Year, Month, Day
            year = d.year
            month = d.strftime("%B").upper()[:3]
            day = d.date().strftime("%d")
            file_name = f"cm{day:0>2}{month}{year}bhav.csv"
            file_path = raw_data_dir / file_name
            if file_path.exists():
                files_to_load.append(file_path)
        return load_multiple_bhavcopy(files_to_load)
    
    else:
        csv_files = [x for x in raw_data_dir.iterdir() if x.suffix == ".csv"]

        # Read all the csv files and concatenate them into one dataframe
        # TODO filter out by end_date
        return load_multiple_bhavcopy(csv_files)

Preprocess bhavcopy data

In [ ]:
# | export
def preprocess(df):
    return (
        df.pipe(lambda x: x[x["SERIES"] == "EQ"])
        .assign(
            DATE=pd.to_datetime(df.TIMESTAMP, format="%d-%b-%Y").dt.date,
            # DAY_OF_WEEK=pd.to_datetime(df.TIMESTAMP, format="%d-%b-%Y").dt.day_name(),
            # WEEK_NUM=pd.to_datetime(df.TIMESTAMP, format="%d-%b-%Y").dt.isocalendar().week,
        )
        .drop(
            columns=[
                "TIMESTAMP",
            ]
        )
        .sort_values(["SYMBOL", "DATE"])
        .reset_index(drop=True)
        # .set_index("DATE")
    )

## Technical analysis utils

Generate SMA

In [ ]:
# | export
# Generate simple moving average data
def get_sma(df_symbol, period=20, metric="CLOSE"):
    if metric.upper() in ["CLOSE", "OPEN", "HIGH", "LOW"]:
        return pd.concat(
            [
                df_symbol,
                pdta.sma(df_symbol[metric], length=period).rename(
                    f"SMA_{period}_{metric.upper()[0]}"
                ),
            ],
            axis=1,
        )
    else:
        raise ValueError("Invalid metric")

Generate Bollinger bands

In [ ]:
# | export
# Generate bollinger bands data
def get_bollinger_bands(df_symbol, period=20, std=2):
    return pd.concat(
        [
            df_symbol,
            pdta.bbands(df_symbol.CLOSE, length=period, std=std).rename(
                columns={
                    f"BBU_{period}_{std:.1f}": f"BBU_{period}_{std}",
                    f"BBM_{period}_{std:.1f}": f"BBM_{period}_{std}",
                    f"BBL_{period}_{std:.1f}": f"BBL_{period}_{std}",
                    f"BBB_{period}_{std:.1f}": f"BBB_{period}_{std}",
                    f"BBP_{period}_{std:.1f}": f"BBP_{period}_{std}",
                }
            ),
        ],
        axis=1,
    )

Get Donchian channels

In [ ]:
# | export
# Generate donchian channel data
def get_donchian(df_symbol, upper=22, lower=66):
    return pd.concat(
        [
            df_symbol,
            pdta.donchian(
                df_symbol.HIGH, df_symbol.LOW, lower_length=66, upper_length=22
            )
            # .rename(
            #     columns={
            #         f"DCL_{lower}_{upper}": f"DONCHIAN_L{lower}",
            #         f"DCU_{lower}_{upper}": f"DONCHIAN_U{upper}"})
            .drop(columns=[f"DCM_{lower}_{upper}"]),
        ],
        axis=1,
    )

Get Supertrend indicator

In [ ]:
# | export
# Generate supertrend data
def get_supertrend(df_symbol, period=12, multiplier=3):
    return pd.concat(
        [
            df_symbol,
            pdta.supertrend(
                df_symbol.HIGH,
                df_symbol.LOW,
                df_symbol.CLOSE,
                length=period,
                multiplier=multiplier,
            )
            .drop(
                columns=[
                    f"SUPERT_{period}_{multiplier:.1f}",
                    f"SUPERTl_{period}_{multiplier:.1f}",
                    f"SUPERTs_{period}_{multiplier:.1f}",
                ]
            )
            .rename(
                columns={
                    f"SUPERTd_{period}_{multiplier:.1f}": f"ST_{period}_{multiplier}"
                }
            ),
        ],
        axis=1,
    )

Get candlestick properties data

In [ ]:
# | export
def add_candle_stats(df_symbol):
    return df_symbol.assign(
        CDL_COLOR=df_symbol.apply(
            lambda x: "green" if x.CLOSE > x.OPEN else "red", axis=1
        ).astype("string"),
        CDL_SIZE=abs(df_symbol.CLOSE - df_symbol.OPEN),
        TOPWICK_SIZE=df_symbol.HIGH - df_symbol[["OPEN", "CLOSE"]].max(axis=1),
        BOTWICK_SIZE=df_symbol[["OPEN", "CLOSE"]].min(axis=1) - df_symbol.LOW,
    )

Add all technical indicator data

In [ ]:
# | export
# Generate all technicals for a symbol data
def add_all_technicals(df_symbol):
    return (
        df_symbol.sort_values(["SYMBOL", "DATE"])
        .reset_index(drop=True)
        # Add SMA
        .pipe(get_sma, period=20, metric="CLOSE")
        .pipe(get_sma, period=20, metric="HIGH")
        .pipe(get_sma, period=44, metric="CLOSE")
        .pipe(get_sma, period=200, metric="CLOSE")
        # Add Bollinger bands
        .pipe(get_bollinger_bands)
        # Add Donchian channel
        # .pipe(get_donchian)
        # Add supertrend data
        # .pipe(get_supertrend, period=12, multiplier=3)
        # .pipe(get_supertrend, period=11, multiplier=2)
        # .pipe(get_supertrend, period=10, multiplier=1)
        # Add candle properties data
        # .pipe(add_candle_stats)
    )

## Process all symbols and add technicals

In [ ]:
# | export
def process_and_save_symbol_data(df):
    df = add_all_technicals(df)
    file_path = processed_data_dir / f"{df.SYMBOL.iloc[-1]}.parquet"
    df.to_parquet(file_path, index=False)
    print(f"Saved {file_path.name}")

In [ ]:
# | export
def rebuild_all_symbols_data():
    df = get_raw_bhavcopy_data()
    df = preprocess(df)
    
    # Recursively delete all files and directories inside the processed data directory
    _ = [
        shutil.rmtree(f) if f.is_dir() else f.unlink()
        for f in processed_data_dir.iterdir()
    ]

    for symbol, df_symbol in df.groupby("SYMBOL"):
        if len(df_symbol) > 200:
            process_and_save_symbol_data(df_symbol)
            

In [ ]:
# | export
def update_all_symbols_data():
    # Define date range
    start_date = (
        pd.read_parquet(processed_data_dir / "INFY.parquet")
        .sort_values(["DATE"])
        .reset_index(drop=True)
        .DATE.iloc[-2]
    )
    end_date = datetime.datetime.now().date()#-datetime.timedelta(days=15)
    print(start_date, end_date)

    # Fetch latest data from NSE
    fetch_bhavcopy_data_for_range(start_date, end_date)

    df = preprocess(get_raw_bhavcopy_data(start_date=start_date))
    new_rows_per_symbol = df.shape[0]/df.SYMBOL.nunique()
    
    if new_rows_per_symbol < 3:
        print("No new data to update")
    else:
        for symbol, df_symbol in df.groupby("SYMBOL"):
            pq = processed_data_dir / f"{symbol}.parquet"
            if pq.exists():
                # Load earlier data
                old_df = pd.read_parquet(pq)
                old_df = old_df.drop(
                    columns=[
                        x
                        for x in old_df.columns
                        if x
                        not in [
                            "SYMBOL",
                            "SERIES",
                            "OPEN",
                            "HIGH",
                            "LOW",
                            "CLOSE",
                            "TOTTRDQTY",
                            "TOTTRDVAL",
                            "TOTALTRADES",
                            "DATE",
                        ]
                    ]
                )
                
                new_df = (
                    pd.concat([old_df, df_symbol])
                    .sort_values(["DATE"])
                    .drop_duplicates(subset=["DATE"], keep="first")
                    .reset_index(drop=True)
                )
                #TODO: new_df has duplicates
                # print(old_df.shape)
                # print(df_symbol.shape)
                process_and_save_symbol_data(new_df)

In [ ]:
update_all_symbols_data()

2023-04-27 2023-04-30
--------------------------------------------------
Fetching data for 3 days
File cm27APR2023bhav.csv.zip already exists.. unzipping
File cm28APR2023bhav.csv.zip already exists.. unzipping
Skipping 2023-04-29 as it is a weekend
Skipping 2023-04-30 as it is a weekend
Bhavcopy data download complete
--------------------------------------------------
No new data to update


In [ ]:
pd.read_parquet(processed_data_dir / "INFY.parquet").tail(20)

,SYMBOL,SERIES,OPEN,HIGH,LOW,CLOSE,TOTTRDQTY,TOTTRDVAL,TOTALTRADES,DATE,SMA_20_C,SMA_20_H,SMA_44_C,SMA_200_C,BBL_20_2,BBM_20_2,BBU_20_2,BBB_20_2,BBP_20_2
1789,INFY,EQ,1392.10,1398.00,1375.85,1377.60,6633366,9.173851e+09,154855,2023-03-28,1434.6025,1451.5550,1507.930682,1507.34400,1344.618938,1434.6025,1524.586062,12.544738,0.183262
1790,INFY,EQ,1379.00,1389.90,1364.55,1383.55,8813966,1.213957e+10,235693,2023-03-29,1429.4025,1444.8500,1504.104545,1506.87775,1340.242725,1429.4025,1518.562275,12.475111,0.242863
1791,INFY,EQ,1404.00,1429.95,1393.90,1427.95,8711975,1.234356e+10,228023,2023-03-31,1425.9975,1441.3050,1501.489773,1506.89500,1342.241106,1425.9975,1509.753894,11.747060,0.511656
1792,INFY,EQ,1435.00,1435.00,1405.15,1410.85,5242764,7.406174e+09,181180,2023-04-03,1422.9325,1438.6250,1499.028409,1506.74650,1341.707902,1422.9325,1504.157098,11.416507,0.425623
1793,INFY,EQ,1400.00,1427.50,1400.00,1423.55,7506334,1.066000e+10,142387,2023-04-05,1420.1450,1435.5100,1496.409091,1506.75325,1343.132197,1420.1450,1497.157803,10.845766,0.522107
1794,INFY,EQ,1416.00,1428.40,1410.35,1421.90,6948465,9.865266e+09,229597,2023-04-06,1415.8675,1430.9100,1493.867045,1506.87675,1350.034626,1415.8675,1481.700374,9.299299,0.545817
1795,INFY,EQ,1423.95,1434.00,1415.00,1427.60,4993788,7.127632e+09,129744,2023-04-10,1412.6125,1427.8100,1491.060227,1507.07825,1356.590443,1412.6125,1468.634557,7.931695,0.633764
1796,INFY,EQ,1428.90,1433.35,1389.00,1406.15,6942255,9.785971e+09,182219,2023-04-11,1408.9000,1424.5600,1487.023864,1507.03825,1362.288156,1408.9000,1455.511844,6.616771,0.470501
1797,INFY,EQ,1413.05,1431.50,1400.00,1428.30,5831169,8.276401e+09,136836,2023-04-12,1406.7375,1422.4200,1483.135227,1506.93025,1368.734528,1406.7375,1444.740472,5.402994,0.783695
1798,INFY,EQ,1409.75,1411.05,1383.00,1389.20,11267238,1.570786e+10,345857,2023-04-13,1404.4425,1419.0575,1479.029545,1506.69000,1368.058654,1404.4425,1440.826346,5.181251,0.290532


In [ ]:
# | hide
nbdev.nbdev_export()
